# Classification with the perceptron and logistic regression

__Individual assignment__

Author of the assignment: Pierre Nugues

__Student name__:

## Objectives

The objectives of this second assignment are to:

1.  Write a linear regression program using gradient descent;
2.  Write linear classifiers using the perceptron algorithm and logistic regression;
3.  Experiment variations of the algorithms;
4.  Evaluate your classifiers;
5.  Experiment with popular tools;
6.  Read a scientific article on optimization techniques and comment it;
7.  Present your code, results, and comments in a short dissertation.

## Overview

The gradient descent is a basic technique to estimate linear discriminant functions. You will first use the gradient descent method to implement linear regression. You will then program the perceptron algorithm. Finally, you will improve the threshold function with the logistic curve (logistic regression). You will try various configurations and study their influence on the learning speed and accuracy.

##  Programming language
As programming language, you will use Python and write your code in this notebook.

You need to have a comprehensive Python distribution such as Anaconda (https://www.anaconda.com/products/individual). This distribution is available on the student computers at the computer science department.
Finally, you start a notebook by typing:

`jupyter lab`

in a terminal window and you select the notebook by clicking on it in the left pane.
You run the pieces of code by typing shift+enter.

## Imports
Imports you may use

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Linear Regression

You will implement the gradient descent method as explained in pages 719--720 in Russell-Norvig and in the slides to compute regression lines. You will implement the stochastic and batch versions of the algorithm.

You must try to do it yourself first. If you encounter difficulties, you also have the solution to this exercise in the section _Solution to linear regression_ below.

### Your implementation of linear regression
You will implement a regression program to predict the counts of _A_'s in a text from the total count of letters. You will apply it on two data sets corresponding to letter counts in the 15 chapters of the French and English versions of _Salammbô_, where the first column is the total count of characters and the second one, the count of A's. 

Start with either French or English and when your program ready, test it on the other language.

In [ ]:
stat_fr = np.array([[36961, 2503],
                      [43621, 2992],
                      [15694, 1042],
                      [36231, 2487],
                      [29945, 2014],
                      [40588, 2805],
                      [75255, 5062],
                      [37709, 2643],
                      [30899, 2126],
                      [25486, 1784],
                      [37497, 2641],
                      [40398, 2766],
                      [74105, 5047],
                      [76725, 5312],
                      [18317, 1215]])

stat_en = np.array([[35680, 2217],
                      [42514, 2761],
                      [15162, 990],
                      [35298, 2274],
                      [29800, 1865],
                      [40255, 2606],
                      [74532, 4805],
                      [37464, 2396],
                      [31030, 1993],
                      [24843, 1627],
                      [36172, 2375],
                      [39552, 2560],
                      [72545, 4597],
                      [75352, 4871],
                      [18031, 1119]])

From the datasets above, tell what is $\mathbf{X}$ and $\mathbf{y}$. Extract:
1. The $\mathbf{X}$ matrix, where you will have a column to model the intercept;
2. The $\mathbf{y}$ vector

from these arrays.

In [ ]:
X_feature = stat_fr[:,0]
print(X_feature)

Y_output = stat_fr[:,1]
print(Y_output)


Scale the arrays so that they fit in the range [0, 1] on the $x$ and $y$ axes. 

In [ ]:
def norm(array):
    temp_array = []
    max_value = np.amax(array)
    for v in array:
        temp_array.append(v/max_value)
    return temp_array

X_scaled = norm(X_feature)
print(X_scaled)

Y_scaled = norm(Y_output)
print(Y_scaled)

#### Gradient descent

Implement the descent functions. You will pass `X`, `y`, the learning rate in the $\alpha$ variable, the initial weight vector in `w`, the tolerance in the $\epsilon$ variable, the maximal number of epochs in `epochs`. You will return `w`.

Batch descent

In [ ]:
# Write your code here
def fit_batch(X, y, alpha, w,
                  epochs=500,
                  epsilon=1.0e-5):
    iterations = 0
    grad  = epsilon + 1
    while(iterations < epochs and np.linalg.norm(grad) > epsilon):
        for x_i, y_i in zip(X,y):
            y_hat = np.dot(w, np.array([1.0, x_i]))
            grad = np.array([1.0, x_i.T] * (y_i - y_hat))
            w += alpha * grad/np.size(X) 
        iterations += 1
    return w

Stochastic descent

In [ ]:
# Write your code here
def fit_stoch(X, y, alpha, w,
                  epochs=500,
                  epsilon=1.0e-5):
    iterations = 0
    idx = list(range(len(X)))
    loss = epsilon +1
    while(iterations < epochs and np.linalg.norm(loss) > epsilon):
        for i in idx:
           
            #print(w)
            loss = (y[i] - ([1,X[i]] @ w))*np.array([1,X[i]])
            w = w + alpha*loss
        iterations += 1
    return w



#### Applying batch descent

Apply the batch descent and print the final weight values 

In [ ]:
# Write your code here
alpha = 1
w_batch = np.zeros((1,2))
w_batch = fit_batch(X_scaled, Y_scaled, alpha, w_batch)
print("Weights batch: {0}".format(w_batch))

Visualize the points of your dataset as well as the regression lines you obtain using matplotlib or another similar program.

In [ ]:
# Write your code here
plt.scatter(X_scaled,Y_scaled)
x_values = np.linspace(0,1,100)
y_values = w_batch[0][1]*x_values + w_batch[0][0]
plt.plot(x_values,y_values)
plt.show()

#### Stochastic descent

In [ ]:
alpha = 1
w_stoch = np.array([0,0])
w_stoch = fit_stoch(X_scaled, Y_scaled, alpha, w_stoch)
print("Weights stoch: {0}".format(w_stoch))

Visualize the points of your dataset as well as the regression lines you obtain using matplotlib or another similar program.

In [ ]:
# Write your code here
plt.scatter(X_scaled,Y_scaled)
x_values = np.linspace(0,1,100)
y_values = w_stoch[1]*x_values + w_stoch[0]
plt.plot(x_values,y_values)
plt.show()

### A solution to linear regression

To help you start this assignment, your instructor wrote two Python notebooks that solve this exercise on linear regression. You can find them here: https://github.com/pnugues/ilppp/tree/master/programs/ch04/python
The first notebook, `gradient_descent.ipynb`, only uses Python and vector operations such as the dot product that are in the `vector.py` file.
The second notebook, `gradient_descent_numpy.ipynb`, uses Numpy. It is more compact, but you need to know a bit of numpy.

To run these programs, download them on your computer as well as the other program in the import list: vector.py

The programs are also available as Python programs from
https://github.com/pnugues/ilppp/tree/master/programs/ch04/python

## Classification
You will use the same data set as for linear regression, but this time to classify a chapter as French or English. Given a pair of numbers corresponding the letter count and count of _A_, you will predict the language:
1. $\mathbf{x} = (35680, 2217)$ $\to$ $y$ = English
2. $\mathbf{x} = (37497, 2641)$ $\to$ $y$ = French

### The dataset
You will use the arrays below:
1. `X` contains the counts of letters and of _A_ s as well as a column of ones for the intercept;
2. `y` contains the classes, where 0 is for English and 1 for French.

In [ ]:
X = [[1.0, 35680.0, 2217.0],
     [1.0, 42514.0, 2761.0],
     [1.0, 15162.0, 990.0],
     [1.0, 35298.0, 2274.0],
     [1.0, 29800.0, 1865.0],
     [1.0, 40255.0, 2606.0],
     [1.0, 74532.0, 4805.0],
     [1.0, 37464.0, 2396.0],
     [1.0, 31030.0, 1993.0],
     [1.0, 24843.0, 1627.0],
     [1.0, 36172.0, 2375.0],
     [1.0, 39552.0, 2560.0],
     [1.0, 72545.0, 4597.0],
     [1.0, 75352.0, 4871.0],
     [1.0, 18031.0, 1119.0],
     [1.0, 36961.0, 2503.0],
     [1.0, 43621.0, 2992.0],
     [1.0, 15694.0, 1042.0],
     [1.0, 36231.0, 2487.0],
     [1.0, 29945.0, 2014.0],
     [1.0, 40588.0, 2805.0],
     [1.0, 75255.0, 5062.0],
     [1.0, 37709.0, 2643.0],
     [1.0, 30899.0, 2126.0],
     [1.0, 25486.0, 1784.0],
     [1.0, 37497.0, 2641.0],
     [1.0, 40398.0, 2766.0],
     [1.0, 74105.0, 5047.0],
     [1.0, 76725.0, 5312.0],
     [1.0, 18317.0, 1215.0]]
y = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
     1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

We visualize it

In [ ]:
x_fr = [x[1] for i, x in enumerate(X) if y[i] == 1]
y_fr = [x[2] for i, x in enumerate(X) if y[i] == 1]
x_en = [x[1] for i, x in enumerate(X) if y[i] == 0]
y_en = [x[2] for i, x in enumerate(X) if y[i] == 0]

In [ ]:
plt.scatter(x_fr, y_fr, color='red')
plt.scatter(x_en, y_en, color='blue')

### Normalize the dataset

Gradient descent algorithms can be very sensitive to the range. Therefore, we normalize the dataset.

In [ ]:
def normalize(observations):
    maxima = [max([obs[i] for obs in observations]) for i in range(len(observations[0]))]
    return ([[obs[i] / maxima[i]
              for i in range(len(observations[0]))] for obs in observations],
            maxima)

In [ ]:
X_norm, maxima = normalize(X)
X_norm

## The Perceptron

1. Write the perceptron program as explained in pages 723--725 in Russell-Norvig and in the slides and run it on your data set. As suggested program structure, use two functions: 
 * `fit(X, y)` that will return `w` (the model). You can choose a stochastic or batch variant;
 * `predict(X, w)` that will return `y_hat`. You can encapsulate these functions in a class and, of course, add more parameters.
2. As a stop criterion, you will use the number of misclassified examples.
3. You will report the parameters you have used and the weight vector

You can use numpy or not. The next cells are just suggested steps. You can implement it your way.

### The `predict(X, w)` function
Write a `predict(X, w)` function that given a matrix of observations $\mathbf{X}$ and a weight vector $\mathbf{w}$ will return a $\mathbf{\hat{y}}$ vector classes (0 or 1)

In [ ]:
# Write your code here
def predict(X, w):
    pred =  X @ w
    if pred > 0:
        return 1
    else:
        return 0


### The `fit(X, y)` function

Write a `fit(X, y)` function that given a matrix of observations $\mathbf{X}$ and a vector of responses $\mathbf{y}$ will return a weight $\mathbf{w}$ vector. You may use the other arguments of the function, notably the number of misclassified examples to define the stop condition.

In [ ]:
# Write your code here
def fit_stoch(X, y,
              epochs=1000,
              max_misclassified=1,
              verbose=True):
    iterations = 0
    X = np.array(X)
    y = np.array(y).reshape(len(y),1)
    w = np.array([0,0,0])
    learn_rate = 1
    while iterations < epochs:
        misclassified = 0
        random.seed(0)
        idx = list(range(len(X)))
        random.shuffle(idx)
        for idx in idx:
            feuture_val = X[idx, :]
            output_val = y[idx]
            pred =  predict(w, feuture_val)
            w = w + learn_rate*(output_val - pred)*feuture_val
            if output_val != pred:
                misclassified += 1
        if misclassified <= max_misclassified:
            break
    return w




### Fitting the dataset

In [ ]:
w = fit_stoch(X_norm, y)
w

In [ ]:
w = [w[i] * maxima[-1] / maxima[i] for i in range(len(w))]
print("Restored weights", w)
w = [w[j] / w[-1] for j in range(len(w))]
print("Weights with y set to 1", w)

### Visualizing the results

In [ ]:
plt.scatter(x_fr, y_fr, color='red')
plt.scatter(x_en, y_en, color='blue')
plt.plot([min(x_fr + x_en), max(x_fr + x_en)],
             [-w[1] * min(x_fr + x_en) - w[0], -w[1] * max(x_fr + x_en) - w[0]])

### Evaluation
Evaluate your perceptron using the leave-one-out cross validation method. You will have to train and run 30 models. In each train/run session, you will train on 29 samples and evaluate on the remaining sample. You have then either a correct or a wrong classification. You will sum these classifications, i.e. the number of correct classifications, to get your final evaluation, for instance 29/30.

In [ ]:
# Write your code here
def leave_one_out_cross_val(X, y, fitting_function):
    data_lenght = len(X)
    w = [0,0,0]
    correctPred = 0
    
    for validationIndex in range(data_lenght):
        tempX = X
        tempy = y
        validation_feature = tempX[validationIndex]
        validation_output = tempy[validationIndex]
        
        tempX = np.delete(tempX,validationIndex,0)
        tempy = np.delete(tempy,validationIndex)
        
        w = fitting_function(tempX,tempy)
        
        prediction = validation_feature @ w
        if (prediction > 0 and validation_output == 1) or (prediction < 0 and validation_output == 0):
            correctPred += 1
            print("Fold {0} on {1}: weights: {2}: Correct".format(validationIndex+1,data_lenght,w))
        else:  
            print("Fold {0} on {1}: weights: {2}: Wrong".format(validationIndex+1,data_lenght,w))
    return correctPred/data_lenght
        


In [ ]:
stoch_accuracy = leave_one_out_cross_val(X_norm, y, fit_stoch)
print('Cross-validation accuracy (stochastic):', stoch_accuracy)

## Logistic Regression
From your perceptron program, implement logistic regression. You can either follow the description from the textbook, S. Russell and R. Norvig, _Artificial Intelligence_, 2010, pages 725--727, or the slides. You can either implement the stochastic or the batch version of the algorithm, or both versions. As stop criterion, you will use either the norm of the gradient or the norm of the difference between two consecutive weight vectors. You will also set a maximal number of epochs. Run the resulting program on your data set.

Write the logistic function, where the $x$ input is a real number.

In [ ]:
# part of delta L(w)
def logistic(x):
    return 1/(1+np.exp(-x))


### The `predict(X, w)` function

Write a `predict_proba()` function that given a matrix of observations $\mathbf{X}$ and a weight vector $\mathbf{w}$ will return a vector of probabilities to belong to class 1: The vector will consist of $P(1|\mathbf{x}_i)$ for all the $i$ rows of $\mathbf{X}$.

In [ ]:
# Write your code here
def predict_proba(X, w):
    return logistic(w @ X)

Write a `predict(X, w)` function that given a matrix of observations $\mathbf{X}$ and a weight vector $\mathbf{w}$ will return the class. You will use `predict_proba()` and set the threshold to belong to class 1 to 0.5.

In [ ]:
# Write your code here
def predict(X, w):
    if predict_proba(X,w) > 0.5:
        return 1
    else:
        return 0


### The `fit(X, y)` function

You will now write the `fit(X, y)` function as with the perceptron. You may call it `fit_stoch(X, y)` or `fit_batch(X, y)`. Use the parameters given in the cell below.

In [ ]:
# Write your code here
def fit_stoch(X, y, alpha=100,
              epochs=1000,
              epsilon=1.0e-4,
              verbose=False):
    X = np.array(X)
    y = np.array(y)
    iterations = 0
    w = np.array([0,0,0])
    idx = list(range(len(X)))
    deltaL = epsilon
    while (iterations < epochs): # and (np.linalg.norm(deltaL) >= epsilon )):
        random.seed(0)
        random.shuffle(idx)
        for  i in idx:
            xColumns = X[i][:]
            #print(xColumns)
            yValue = y[i]
            deltaL = xColumns*(yValue - predict(xColumns,w))
            w = w + alpha*deltaL
        iterations += 1
    return w
            

In [ ]:
w = fit_stoch(X_norm, y, verbose=True)
w

In [ ]:
w = [w[i] / maxima[i] for i in range(len(w))]
print("Restored weights", w)
w = [w[j] / w[-1] for j in range(len(w))]
print("Weights with y set to 1", w)

### Visualizing the results

In [ ]:
plt.scatter(x_fr, y_fr, color='red')
plt.scatter(x_en, y_en, color='blue')
plt.plot([min(x_fr + x_en), max(x_fr + x_en)],
         [-w[1] * min(x_fr + x_en) - w[0], -w[1] * max(x_fr + x_en) - w[0]])
plt.show()

### Evaluation
Evaluate your logistic regression using the leave-one-out cross validation method as with the perceptron

In [ ]:
def leave_one_out_cross_val(X, y, fitting_function):
    data_lenght = len(X)
    w = [0,0,0]
    correctPred = 0

    for validationIndex in range(data_lenght):
        tempX = X
        tempy = y
        validation_feature = tempX[validationIndex]
        validation_output = tempy[validationIndex]

        tempX = np.delete(tempX,validationIndex,0)
        tempy = np.delete(tempy,validationIndex)

        w = fitting_function(tempX,tempy)

        prediction = validation_feature @ w
        if (prediction > 0 and validation_output == 1) or (prediction < 0 and validation_output == 0):
            correctPred += 1
            print("Fold {0} on {1}: weights: {2}: Correct".format(validationIndex+1,data_lenght,w))
        else:  
            print("Fold {0} on {1}: weights: {2}: Wrong".format(validationIndex+1,data_lenght,w))
    return correctPred/data_lenght
        



In [ ]:
stoch_accuracy = leave_one_out_cross_val(X, y, fit_stoch)
print('Cross-validation accuracy (batch):', stoch_accuracy)

## Visualizing the logistic surface

In [ ]:
def plot_logistic_surf(x_range, y_range, w_opt):
    z_axis = np.array([[0.0] * len(y_range) for i in range(len(x_range))])
    x_axis, y_axis = np.meshgrid(x_range, y_range)
    z_axis = z_axis.reshape(x_axis.shape)

    # We compute the probability surface as a function of x and y
    for i in range(len(x_range)):
        for j in range(len(y_range)):
            z_axis[j, i] = logistic(np.dot([1, x_range[i], y_range[j]], w_opt))
    return x_axis, y_axis, z_axis

In [ ]:
x_range = np.linspace(0, 100000, 200)
y_range = np.linspace(0, 10000, 200)
#w = [2.073225839414742, -0.049125455233437906, 0.7440143556104162]

x_axis, y_axis, z_axis = plot_logistic_surf(x_range, y_range, w)

fig = plt.figure()
ax = Axes3D(fig)
# ax = fig.gca(projection='3d')

surf = ax.plot_surface(y_axis, x_axis, z_axis, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, alpha=0.2)
fig.colorbar(surf, shrink=0.5, aspect=5)
# We plot the observations
for x, y_class in zip(X, y):
    if y_class == 1:
        ax.scatter(x[2], x[1], y_class, color='green', marker='x')
    else:
        ax.scatter(x[2], x[1], y_class, color='red', marker='x')

ax.elev = 30 
ax.azim = -150
plt.show()

## Programming logistic regression with popular APIs
Should you use logistic regression in a project, you will probably resort to existing libraries. In the next cells, you will apply the logistic regression classification with two popular APIs:
1. sklearn
2. Keras

`sklearn` is included in anaconda.
You will install the rest with:
```
pip install --upgrade keras tensorflow tensorflow-addons 
```
You will read and run the code in the cells below.

All these APIs are built on numpy and we convert the dataset into numpy if you have not done it already

In [ ]:
X = np.array(X)
X_norm = np.array(X_norm)
y = np.array(y)

They also handle the intercept so we do not need the first column of ones.

In [ ]:
X = X[:, 1:]
X_norm = X_norm[:, 1:]

### sklearn
Using the dataset of English and French datapoints, we apply logistic regression with the sklearn API. We need the `LogisticRegression` class, the fit() and predict() functions. The weights are in the `coef_` field.

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model = model.fit(X, y)
model.coef_

We predict the classes of the $\mathbf{X}$ with the `predict()` function

In [ ]:
model.predict(X)

We predict the class probabilities of the $\mathbf{X}$ with the `predict_proba()` function

In [ ]:
model.predict_proba(X)

### Keras
Using the dataset of English and French datapoints, we apply logistic regression with Keras. We need the `Sequential` and `Dense` classes.

In [ ]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense

In [ ]:
# Write your code here
model = Sequential()
model.add(Dense(1, input_dim=2, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model.summary()

We fit the model

In [ ]:
model.fit(X, y, epochs=2500, batch_size=4, verbose=0)

In [ ]:
y

In [ ]:
model.predict(X)

In [ ]:
(model.predict(X) > 0.5).astype('int32')

If you do not obtain a correct classification, rerun the training with more epochs.

## Reading
You will read the article *An overview of gradient descent optimization algorithms* by Ruder (2017) and you will outline the main characteristics of all the optimization algorithms the author describes. This part should be of about one to two pages. Link to the article: https://arxiv.org/abs/1609.04747

If you understand French, or using Google translate, you may also want to read the original article on gradient descent by Cauchy here:  https://gallica.bnf.fr/ark:/12148/bpt6k2982c/f540.item.


### Report

The assignment must be documented in the report, which should contain the following:

*   The name of the author, the title of the assignment, and any relevant information on the front page;
*   A presentation of the assignment and the possible improvements you would have brought;
*   A presentation of your implementation;
*   A print-out of the example set(s) and the resulting weight vectors;
*   Comments on the results you have obtained, including your cross validation;
*   A short dissertation on the optimization algorithms from Ruder's paper.

Please, typeset and format your report consistently. You must use Latex. Documents written using MS Word or any similar format will not be considered.

You may have a look at the code in the textbook code repository (or any other implementations), but the code you hand in must be your work.

## Submission
Submit the notebook and the report to Canvas (two files). Do not include the code printout in the report, but only comments on its interesting parts. You will submit the notebook as a separate file.